In [19]:
import concurrent.futures
import hashlib
import json
import os
import random
import threading
import traceback
from datetime import date, datetime
from decimal import Decimal
from typing import List, Literal, Optional

# import base64
# import gzip
# import uuid
import boto3
import requests
from amazondax import AmazonDaxClient
from aws_lambda_powertools import Logger, Tracer
from pydantic import BaseModel, Field, ValidationError, validator

In [20]:
logger = Logger(log_uncaught_exceptions=True)

In [21]:
class Destination(BaseModel):
    name: str
    lat: Optional[float]
    lon: Optional[float]


class Recommendations(BaseModel):
    recommendations: List[Destination]
    

In [16]:
def convert_floats_to_decimals(item):
    if isinstance(item, float):
        return Decimal(str(item))
    elif isinstance(item, dict):
        return {k: convert_floats_to_decimals(v) for k, v in item.items()}
    elif isinstance(item, list):
        return [convert_floats_to_decimals(i) for i in item]
    else:
        return item


def write_experience_data(
    recommendations: list, experience: str, destination: str = None
):
    try:
        if not destination:
            destination = "no_destination"
        item = {
            "Experience": experience,
            "Destination": destination,
            "Recommendations": recommendations,
        }
        # experience_cache_dynamo_table.put_item(Item=item)
        logger.info(
            "Successfully cached recommendations for destination = %s and experience = %s",
            destination,
            experience,
        )
    except Exception:
        logger.error(
            "Error writing to Dynamo table: %s", experience_cache_dynamo_table_name
        )
        traceback.print_exc()
        raise ValueError("INTENT_SEARCH_API_ERROR_500") from None


def read_experience_data(experience: str, destination: str = None):
    try:
        response = experience_cache_dynamo_table.get_item(
            Key={"Experience": experience, "Destination": destination}
        )
        if "Item" in response:
            logger.info()
            return response["Item"]
        else:
            logger.info(
                "No item found for %s in %s in experience cache.",
                experience,
                destination,
            )
            return None
    except Exception:
        logger.error(
            "Error reading from Dynamo table: %s", experience_cache_dynamo_table_name
        )
        traceback.print_exc()
        raise ValueError("INTENT_SEARCH_API_ERROR_500") from None


def get_recommendations_from_gemini(experience, destination):
    recommendations = [
        {"name": "Waveland Bowl", "lat": 41.9583, "lon": -87.6638},
        {"name": "Pinstripes", "lat": 41.8781, "lon": -87.6298},
        {
            "name": "The lanes at the Chicago Athletic Association",
            "lat": 41.8836,
            "lon": -87.6256,
        },
        {"name": "The Brunswick Zone", "lat": 41.9795, "lon": -87.7839},
        {"name": "The  Logan Square Auditorium", "lat": 41.9252, "lon": -87.7036},
    ]
    try:
        Recommendations(recommendations=recommendations)
    except ValidationError as e:
        logger.error("Request validation error: %s", e)
        traceback.print_exc()
        raise ValueError("INTENT_SEARCH_API_ERROR_400") from None
    return recommendations


def get_recommendations(experiences: list, destinations: list):
    if not experiences:
        logger.error("No experiences found in the query.")
        raise ValueError("INTENT_SEARCH_API_ERROR_500") from None

    # only support single experience/destination
    experience = experiences[0]["name"]
    destination = destinations[0]["name"] if destinations else "no_destination"
    print(experience,destination)
    # recommendations_in_cache = read_experience_data(experience, destination)
    recommendations_in_cache=None
    if recommendations_in_cache:
        logger.info(
            "Found recommendations for %s in %s in cache: %s",
            experience,
            destination,
            recommendations_in_cache,
        )
        return recommendations_in_cache
    else:
        recommendations = get_recommendations_from_gemini(experience, destination)
        logger.info(
            "Recommendations returned by Gemini: %s",
        )
        recommendations = convert_floats_to_decimals(recommendations)
        write_experience_data(experience, destination, recommendations)
        logger.info("New experience added to cache.")

In [22]:
experiences = [{'name':'bowling'}]
destinations = [{'name':'chicago'}]
# get_recommendations(experiences, destinations)

In [24]:
recommendations = [
        {"name": "Waveland Bowl", "lat": 41.9583, "lon": -87.6638},
        {"name": "Pinstripes", "lat": 41.8781, "lon": -87.6298},
        {
            "name": "The lanes at the Chicago Athletic Association",
            "lat": 41.8836,
            "lon": -87.6256,
        },
        {"name": "The Brunswick Zone", "lat": 41.9795, "lon": -87.7839},
        {"name": "The  Logan Square Auditorium", "lat": 41.9252, "lon": -87.7036},
    ]


In [25]:
r = convert_floats_to_decimals(recommendations)

In [26]:
r

[{'name': 'Waveland Bowl',
  'lat': Decimal('41.9583'),
  'lon': Decimal('-87.6638')},
 {'name': 'Pinstripes', 'lat': Decimal('41.8781'), 'lon': Decimal('-87.6298')},
 {'name': 'The lanes at the Chicago Athletic Association',
  'lat': Decimal('41.8836'),
  'lon': Decimal('-87.6256')},
 {'name': 'The Brunswick Zone',
  'lat': Decimal('41.9795'),
  'lon': Decimal('-87.7839')},
 {'name': 'The  Logan Square Auditorium',
  'lat': Decimal('41.9252'),
  'lon': Decimal('-87.7036')}]